In [38]:
import sys
sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *
from pythreejs import *

In [39]:
mss = MassSpringSystem3d()
mss.gravity = (0,-9.81,0)

massVal = 1.0
length = 2
totalForce = 10

height = length * 6**0.5 / 3 # tetrahedon height
radius = length / math.sqrt(3)

# Force for m1:
f_vec1 = (totalForce, 0, 0)

# Force for m2: Position (r*sin60, -r*cos60) -> Vector (-r*cos60, -r*sin60) 
f_vec2 = (totalForce * (-math.cos(math.pi/3)), totalForce * (-math.sin(math.pi/3)), 0)

# Force for m3: Position (-r*sin60, -r*cos60) -> Vector (-r*cos60, +r*sin60)
f_vec3 = (totalForce * (-math.cos(math.pi/3)), totalForce * (math.sin(math.pi/3)), 0)

top = mss.add (Fix((0,0,0)))
#Point 1
m1 = mss.add(Mass(massVal, (0, radius, -height), f_vec1, (0,0,0)))
# Point 2: 120° rotated (x = r * sin(120°); y = r * cos(120°))
m2 = mss.add(Mass(massVal, (radius * math.sin(math.pi * 2/3), radius * math.cos(math.pi * 2/3), -height), (f_vec2), (0,0,0)))
# Point 3: 240° rotated (x = r * sin(240°); y = r * cos(240°))
m3 = mss.add(Mass(massVal, (radius * math.sin(math.pi * 4/3), radius * math.cos(math.pi * 4/3), -height), (f_vec3), (0,0,0)))

mss.addConstraint(top, m1)
mss.addConstraint(top, m2)
mss.addConstraint(top, m3)
mss.addConstraint(m1, m2)
mss.addConstraint(m2, m3)
mss.addConstraint(m3, m1)

5

In [40]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
springs = None
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 
if springpos:
    springgeo = LineSegmentsGeometry(positions=springpos)
    m2 = LineMaterial(linewidth=3, color='cyan')
    springs = LineSegments2(springgeo, m2)    

constraintpos = []
constraints = None
for c in mss.distanceConstraints:
    pA = mss[c.connectors[0]].pos
    pB = mss[c.connectors[1]].pos
    constraintpos.append ([ pA, pB ] ) 
if constraintpos:
    constraintgeo = LineSegmentsGeometry(positions=constraintpos)
    m3 = LineMaterial(linewidth=3, color='orange')
    constraints = LineSegments2(constraintgeo, m3)

axes = AxesHelper(1)

In [41]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, constraints, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [42]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    if springpos:
        springs.geometry = LineSegmentsGeometry(positions=springpos)
    constraintpos = []
    for c in mss.distanceConstraints:
        pA = mss[c.connectors[0]].pos
        pB = mss[c.connectors[1]].pos
        constraintpos.append ([ pA, pB ] ) 
    if constraintpos:
        constraints.geometry = LineSegmentsGeometry(positions=constraintpos)
    sleep(0.01)

KeyboardInterrupt: 